<a href="https://colab.research.google.com/github/alphavergopi/flask_apps/blob/master/NoMask_yolov3_Darknet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive

/content/drive/My Drive


In [3]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 14263, done.
remote: Total 14263 (delta 0), reused 0 (delta 0), pack-reused 14263
Receiving objects: 100% (14263/14263), 12.63 MiB | 5.62 MiB/s, done.
Resolving deltas: 100% (9774/9774), done.
Checking out files: 100% (2014/2014), done.


In [4]:
%cd darknet

/content/drive/My Drive/darknet


In [5]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

print("Building. . . It might take 2-3 minutes")

!make &> build_log.txt

Building. . . It might take 2-3 minutes


In [6]:
!wget "https://www.dropbox.com/s/6gewe947ake1g95/kaggle_and_no-mask_dataset.zip?dl=1" -O kaggle_and_no-mask_dataset.zip
!unzip kaggle_and_no-mask_dataset.zip -d no-mask_dataset

--2020-08-19 14:52:26--  https://www.dropbox.com/s/6gewe947ake1g95/kaggle_and_no-mask_dataset.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/6gewe947ake1g95/kaggle_and_no-mask_dataset.zip [following]
--2020-08-19 14:52:26--  https://www.dropbox.com/s/dl/6gewe947ake1g95/kaggle_and_no-mask_dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc491e9d38f12b300507f1bf401.dl.dropboxusercontent.com/cd/0/get/A9yJTX9U-h-YiV6CUYrGqLdibECVbCXnmIXNpoFwMZXQVHGiaj3Zqi1Y5SI4Z0dENLKyrn3pkcBLyaEkrK_rFNiSgkAvqAP2t72ROrPDRuXMMvUK9sjzHvCDpZ8q39CGN8U/file?dl=1# [following]
--2020-08-19 14:52:27--  https://ucc491e9d38f12b300507f1bf401.dl.dropboxusercontent.com/cd/0/get/A9yJTX9U-h-YiV6CUYrGqLdibECVbCXnmIXNpoFwMZXQVHGiaj3Zqi1

In [ ]:
import random
import os
import subprocess
import sys

image_dir = "./no-mask_dataset"
f_val = open("no-mask_data_test.txt", 'w')
f_train = open("no-mask_data_train.txt", 'w')

path, dirs, files = next(os.walk(image_dir))
data_size = len(files)

ind = 0
data_test_size = int(0.2 * data_size)
test_array = random.sample(range(data_size), k=data_test_size)

for f in os.listdir(image_dir):
    if(f.split(".")[-1] == "jpg"):
        ind += 1
        
        if ind in test_array:
            f_val.write(image_dir+'/'+f+'\n')
        else:
            f_train.write(image_dir+'/'+f+'\n')

f_train.close()
f_val.close()

In [7]:
!wget "https://www.dropbox.com/s/18dwbfth7prbf0h/darknet53.conv.74?dl=1" -O darknet53.conv.74

--2020-08-19 14:54:05--  https://www.dropbox.com/s/18dwbfth7prbf0h/darknet53.conv.74?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/18dwbfth7prbf0h/darknet53.conv.74 [following]
--2020-08-19 14:54:05--  https://www.dropbox.com/s/dl/18dwbfth7prbf0h/darknet53.conv.74
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc14e0959a841b19629bec70a8ef.dl.dropboxusercontent.com/cd/0/get/A9wKc1l1OW_ZcICOqYHzu111OH_WI1J6bJsgAUO3RtLT0XNiwk3RBbo0LxaWQ0Tb7BDlXtExoh1Z1XT9pXl6Kg5Q8VJjTQqM3dWexyIahskbAoGLsCLAdb2AiZM1kXz7wVk/file?dl=1# [following]
--2020-08-19 14:54:06--  https://uc14e0959a841b19629bec70a8ef.dl.dropboxusercontent.com/cd/0/get/A9wKc1l1OW_ZcICOqYHzu111OH_WI1J6bJsgAUO3RtLT0XNiwk3RBbo0LxaWQ0Tb7BDlXtExoh1Z1XT9pXl6Kg5Q8VJjTQ

Need to create:

yolov3-no-mask-test.cfg  
yolov3-no-mask-train.cfg  
yolov3-no-mask-setup.data 